In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score,roc_auc_score, classification_report
import warnings
warnings.filterwarnings('ignore')
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from datasets import Dataset
import torch
import torch.nn.functional as F

In [2]:
import torch
print(torch.cuda.is_available())  # Should print: True
print(torch.cuda.get_device_name())  # Should print your GPU name (e.g., 1660 Ti)

True
NVIDIA GeForce GTX 1660 Ti


In [3]:
# 1. Load your SnapFood dataset
df = pd.read_csv("https://raw.githubusercontent.com/samyvivo/Snap_Food_Sentiment_Analysis_fa/refs/heads/main/snappfood_sentiment_train.csv")

In [4]:
df = df[["comment", "label"]].dropna()
df["label"] = np.where(df["label"] == "SAD", 0, 1)
df["label"] = df["label"].astype(int)

In [5]:
# 2. Split dataset
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["comment"].tolist(), df["label"].tolist(), test_size=0.2, random_state=42)

In [6]:
# 3. Tokenize using BERT tokenizer
model_name = "HooshvareLab/bert-fa-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)

In [7]:
# 4. Convert to HuggingFace Dataset format
train_dataset = Dataset.from_dict({
    "input_ids": train_encodings["input_ids"],
    "attention_mask": train_encodings["attention_mask"],
    "label": train_labels
})
test_dataset = Dataset.from_dict({
    "input_ids": test_encodings["input_ids"],
    "attention_mask": test_encodings["attention_mask"],
    "label": test_labels
})

In [8]:
# 5. Load pre-trained BERT model
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/bert-fa-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# 6. Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    learning_rate=2e-5,
    logging_dir="./logs",
    logging_steps=500,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True
)

In [10]:
# 7. Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [11]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Get predicted class labels
    preds = torch.argmax(torch.tensor(logits), axis=1)
    
    # Apply softmax to get probabilities
    probs = F.softmax(torch.tensor(logits), dim=1)[:, 1]  # Probability of class 1

    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds),
        "roc_auc": roc_auc_score(labels, probs)
    }

In [12]:
# 8. Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [13]:
# 9. Train the model
trainer.train(resume_from_checkpoint=True)

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# Save model
trainer.save_model(r"C:\Users\samyh\Desktop\BERT/snapfood-bert-model")

# Save tokenizer
tokenizer.save_pretrained(r"C:\Users\samyh\Desktop\BERT/snapfood-bert-model")

NameError: name 'trainer' is not defined